In [1]:
!pip install 'bs4==0.0.1' 'lxml==4.9.2' 'googlemaps>=4.10.0' 'python-dotenv'


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.parse import unquote
from dotenv import dotenv_values
import json
import time
import googlemaps

env = dotenv_values()

# API key for places, directions, geocode and another to maps js
gmaps = googlemaps.Client(key=env["GOOGLE_KEY"])

locations = []
addrs = []
geo = []
names = []
url = []
food = []
food_desc = []
contact = []
work_hours = []
images = []

def get_google(name, query, retry=3):

    try:
        result = gmaps.geocode(unquote(query))
        if len(result) > 1:
            place = gmaps.places(unquote(name))["results"]
            if len(place) == 1:
                result = place
        if not result: raise Exception()
        return result
    except Exception as e:
        if retry:
            time.sleep(1)
            return get_google(name, query, retry-1)
        else:
            print(f"query error {query}")
            print(e)
            return False

def request(url):
    r = Request(url, headers={'User-Agent' : "Magic Browser"})
    resp = urlopen(r)
    return BeautifulSoup(resp, "lxml")

for i in range(1, 90):
    soup = request(f"https://comidadibuteco.com.br/butecos/page/{i}/")
    inner = soup.findAll('div', {"class": "item"})
    for div in inner:
        name = div.find('h2').text
        links = div.findAll('a')
        if name and (len(links) == 2):
            u = links[0].get('href')
            s = request(u)
            data = s.find('div', {"class": "section-text"})
            image = s.find('main').find('div', {"class": "row"}).find('img').get('src')
            addr = ''
            snack_details = ''
            phone = ''
            work = ''
            try:
                place = data.findAll('p')
                next_p = ''
                sanity_data = data.text.replace('\n', ' ').replace('\t', ' ').replace('\r', '').replace('\xa0', '')
                try:
                    snack = data.findAll('b')[0].text.replace('\n', ' ').replace('\t', ' ').replace('\r', '').replace('\xa0', '')
                    addr_index = sanity_data.index('Endereço')
                except Exception as e:
                    raise e
                indexes = {'Endereço': addr_index, 'Complemento': None, 'Telefone': None, 'Horario': None}
                if 'Complemento' in sanity_data:
                    comp_index = sanity_data.index('Complemento')
                    indexes['Complemento'] = comp_index
                if 'Telefone' in sanity_data:
                    phone_index = sanity_data.index('Telefone')
                    indexes['Telefone'] = phone_index
                if 'Horario' in sanity_data:
                    work_index = sanity_data.index('Horario')
                    indexes['Horario'] = work_index

                indexes = {k: v for k, v in indexes.items() if v is not None}
                old_label, old_idx = sorted(indexes.items(), key=lambda item: item[1])[0]
                snack_details = sanity_data[len(snack)+1:old_idx]

                results = {k: '' for k in indexes.keys()}
                for label, idx in sorted(indexes.items(), key=lambda item: item[1]):
                    if old_idx != idx:
                        results[old_label] = sanity_data[old_idx+len(old_label): idx].split(':')[-1]
                    old_idx, old_label = (idx, label)
                results[old_label] = sanity_data[old_idx+len(old_label):].split(':')[-1]
                
                addr = results.get('Endereço', '')
                comp = results.get('Complemento', '')
                phone = results.get('Telefone', '')
                work = results.get('Horario', '')
                    
            except Exception as e:
                print(f'fallback to maps link on {name}')
                print(e)

                addr = links[1].get('href').split('query=')[-1]
                
            if not addr:
                print(f"no addr found for {name}")
                continue
            geocode_result = get_google(name, addr.replace('|', '-'))
            if geocode_result:
                geo.append(geocode_result[0]["geometry"]["location"])
            else:
                print(f"no place found for {name}")
                geo.append({})
            url.append(u)
            addrs.append(f'{addr} - {comp}'.strip())
            names.append(name.strip())
            food.append(snack.strip())
            food_desc.append(snack_details.strip())
            contact.append(phone.strip())
            images.append(image)
            work_hours.append(work.strip().replace('hQ', 'h | Q').replace('hT', 'h | T').replace('hS', 'h | S').replace('hD', 'h | D').replace('h Q', 'h | Q').replace('h T', 'h | T').replace('h S', 'h | S').replace('h D', 'h | D'))
        else:
            print(f'page {i} with name {name} failed')
            continue
    print(f"foi página {i}")
    time.sleep(10)
for i in range(len(names)):
    locations.append({
        "name": names[i],
        "addrs": addrs[i],
        "geo": geo[i],
        "url": url[i],
        "img": images[i],
        "food": food[i],
        "food_desc": food_desc[i],
        "contact": contact[i],
        "work_hours": work_hours[i]
    })

foi página 1
foi página 2
foi página 3
foi página 4
foi página 5
foi página 6
foi página 7
foi página 8
foi página 9
foi página 10
foi página 11
foi página 12
foi página 13
foi página 14
foi página 15
foi página 16
foi página 17
foi página 18
foi página 19
foi página 20
foi página 21
foi página 22
foi página 23
foi página 24
foi página 25
foi página 26
foi página 27
foi página 28
foi página 29
foi página 30
foi página 31
foi página 32
foi página 33
foi página 34
foi página 35
foi página 36
foi página 37
foi página 38
foi página 39
foi página 40
foi página 41
foi página 42
foi página 43
foi página 44
foi página 45
foi página 46
foi página 47
foi página 48
foi página 49
foi página 50
foi página 51
foi página 52
foi página 53
foi página 54
foi página 55
foi página 56
foi página 57
foi página 58
foi página 59
foi página 60
foi página 61
foi página 62
foi página 63
foi página 64
foi página 65
foi página 66
foi página 67
foi página 68
foi página 69
foi página 70
foi página 71
foi página 72
f

In [3]:
locations

[{'name': '1960 Bar',
  'addrs': 'Rua 237, 307 | Setor Coimbra, Goiânia – GO  -  Quadra 45, Lote 01',
  'geo': {'lat': -16.6810272, 'lng': -49.2825398},
  'url': 'https://comidadibuteco.com.br/buteco/1960-bar/',
  'img': 'https://cdb-static-files.s3.amazonaws.com/wp-content/uploads/2025/03/14005511/1960-bar.jpg',
  'food': 'Cone de Prata',
  'food_desc': 'Cone de pastel, recheado com escabeche de carne de sol desfiada e maionese aioli. Acompanha molho da casa.',
  'contact': '6292362753',
  'work_hours': 'Segunda, das 17h às 22h | Terça, das 17h às 22h | Quinta, das 17h às 22h | Quarta, das 17h às 22h | Sexta, das 17h às 22h | Sábado, das 11h30 às 16h |'},
 {'name': '24 Do 4',
  'addrs': 'Dr. Thomás, 151 | Nossa Sra das Graças, Manaus – AM  -   Loja 01',
  'geo': {'lat': -3.0991763, 'lng': -60.01549209999999},
  'url': 'https://comidadibuteco.com.br/buteco/24-do-4/',
  'img': 'https://cdb-static-files.s3.amazonaws.com/wp-content/uploads/2025/03/15141933/24-do-4.jpg',
  'food': 'Chilli 

In [4]:
with open("places.json", "w") as f:
    f.write(json.dumps(locations, ensure_ascii=False))

In [5]:
len(names)

1063

In [6]:
len(geo)

1063

In [7]:
len(url)

1063

In [8]:
len(food)

1063

In [9]:
len(food_desc)

1063